## 1.10

In [7]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [8]:
a = [1,2,3,4,5,6,1]

In [36]:
list(dedupe(a))

['look',
 'into',
 'my',
 'eyes',
 'the',
 'not',
 'around',
 "don't",
 "you're",
 'under']

## 1.11 命名切片

避免使用硬切片

In [11]:
a = slice(2,4)
items = [0,1,2,3,4,5,6]
items[a]

[2, 3]

In [12]:
a = slice(2,6,2)
items[a]

[2, 4]

In [20]:
a = slice(2,50,2)
a.indices(len(items))

(2, 7, 2)

In [23]:
for i in range(*a.indices(len(items))):
    //这里 a.indices()返回的是一个元组 用*可以取得里面的元素
    //这里恰好有三个元素组成了range(start,end,step)
    print(items[i])
    

2
4
6


## 1.12 序列中出现次数最多的元素

In [34]:
from collections import Counter
# 需要制表或计数数据的场合都很有用
words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]
morewords = ['why','are','you','not','looking','in','my','eyes']

words_count = Counter(words)
top_three = words_count.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


In [35]:
a = Counter(words)
b = Counter(morewords)
print(a + b)
print(a - b)

Counter({'eyes': 9, 'the': 5, 'look': 4, 'my': 4, 'into': 3, 'not': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1, 'why': 1, 'are': 1, 'you': 1, 'looking': 1, 'in': 1})
Counter({'eyes': 7, 'the': 5, 'look': 4, 'into': 3, 'my': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1})


## 1.13 通过某个关键字排序一个字典列表

In [43]:
from operator import itemgetter
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}
]

# 将itemgetter当作key
rows_by_name = sorted(rows, key = itemgetter('fname'))
rows_by_uid = sorted(rows, key = itemgetter('uid'))

# itemgetter可以接受多个keys
rows_by_flname = sorted(rows, key = itemgetter('fname','lname'))

# 可以用于max min
max_rows = max(rows, key = itemgetter('uid'))

# lambda 表达式也能胜任

rows_by_uid = sorted(rows, key = lambda x: x['uid'])

print(rows_by_name)
# print(rows_by_uid)
# print(rows_by_flname)
# print(max_rows)


[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


In [44]:
# itemgetter实现方法
def myitemgetter(*items):
    if len(items) == 1:
        item = items[0]
        def g(obj):
            return obj[item]
    else:
        def g(obj):
            return tuple(obj[item] for item in items)
    return g

In [45]:
g = myitemgetter(1,3,5)
a = [1,2,3,4,5,6,7]
g(a)

(2, 4, 6)

## 1.14 排序不支持原生比较的对象

In [57]:
'''
理解sorted()中关键字key的作用，key可以传入一个callable对象
callable对象对传入的每个对象 返回一个值，这个值会被sorted用来排序
'''
class User:
    def __init__(self, user_id):
        self.user_id = user_id


    def __repr__(self):
        return 'User({})'.format(self.user_id)


def sort_notcompare():
    users = [User(23), User(3), User(99)]
    print(users)
    # 使用lambda 将id用作排序对象
    print(sorted(users, key=lambda u: u.user_id))
    
sort_notcompare()


[User(23), User(3), User(99)]
[User(3), User(23), User(99)]


In [60]:
# 使用operator.attrgetter()
users = [User(23), User(3), User(99)]
from operator import attrgetter
sorted(users, key = attrgetter('user_id'))

# 使用operator.attrgetter()会运行的更快，并且可以传入多个属性
by_name = sorted(users, key=attrgetter('last_name', 'first_name'))


[User(3), User(23), User(99)]

In [53]:
# callable 演示
class mycall:
    def __call__(self):
        print("callable")
a = mycall() # 类callable
a() # 实现了__call__ 的类的实例也是callable
callable(a)

callable


True

## 1.15

In [67]:
rows = [
    {'address': '5412 N CLARK', 'date': '07/01/2012'},
    {'address': '5148 N CLARK', 'date': '07/04/2012'},
    {'address': '5800 E 58TH', 'date': '07/02/2012'},
    {'address': '2122 N CLARK', 'date': '07/03/2012'},
    {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
    {'address': '1060 W ADDISON', 'date': '07/02/2012'},
    {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
    {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

from operator import itemgetter
from itertools import groupby
'''
groupy()函数扫描整个序列并且查找连续相同值
（或者根据指定 key 函数返回值相同）的元素序列。 
在每次迭代的时候，它会返回一个值和一个迭代器对象， 
这个迭代器对象可以生成元素值全部等于上面那个值的组中所有对象。
但是groupy()仅仅检查连续的元素，因此需要先排序

'''
rows.sort(key = itemgetter('date'))

for date,items in groupby(rows, key = itemgetter('date')):
    print(date)
    for i in items:
        print(' ', i)

07/01/2012
  {'address': '5412 N CLARK', 'date': '07/01/2012'}
  {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800 E 58TH', 'date': '07/02/2012'}
  {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
  {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148 N CLARK', 'date': '07/04/2012'}
  {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


## 1.16 过滤序列元素

In [80]:
value1 = [1,2,3,-5,-19,0]
# 1、使用列表推倒或生成式表达式
pos = [n*n if n > 0 else 0 for n in value1]

# 2、使用filter()函数 可以处理异常 更强大
value2 = ['1',2,'3','-','N/A',0]
def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False
ivals = list(filter(is_int, value2))
print(ivals)

# 3、itertools.compress()
# 它以一个 iterable 对象和一个相对应的 Boolean 选择器序列作为输入参数。 
# 然后输出 iterable 对象中对应选择器为 True 的元素。

addresses = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
]
counts = [ 0, 3, 10, 4, 1, 7, 6, 1]
from itertools import compress
more5 = [n > 5 for n in counts]
list(compress(addresses,more5))


['1', 2, '3', 0]


['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']

## 1.17 从字典中提取子集

In [99]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}
# 1、字典推倒式 更快
p1 = {key: value for key, value in prices.items() if value > 200}
# Make a dictionary of tech stocks

# 2、dict()函数
p2 = dict((key, value) for key, value in prices.items() if value > 200)

print(p1)
print(p2)

{'AAPL': 612.78, 'IBM': 205.55}
{'AAPL': 612.78, 'IBM': 205.55}


## 1.18 映射名称到序列元素

In [125]:
# 普通元组通过下标访问很不方便 命名元组解决了这个问题
from collections import namedtuple
Stock = namedtuple('Stock',['name','shares','price'])
def compute_cost(records):
    totol = 0
    for rec in records:
        s = Stock(*rec)
        total += s.shares + s.price
    return total
s = Stock('acm',10,14)
s = s._replace(price = 1)
s = Stock('as',None,None)
def dict_to_stock(a):
    return s._replace(**a)
s
a = {"shares":13,"price":3}
dict_to_stock(a)


Stock(name='as', shares=13, price=3)

In [ ]:
def isAnagram(s1, s2):
    s1.sort()
    s2.sort()
    if (s1 == s2):
        print('is an anagram')
    else:
        print('is not an anagram')

s1 = input()
s2 = input()
s1 = s1.split(' ')
s2 = s2.split(' ')

isAnagram(s1, s2)